In [1]:
import torch.optim as optim
import pandas as pd
import torch
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torchvision.transforms as T

In [2]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms as T

class CustomCSVDetectionDataset(Dataset):
    def __init__(self, csv_file, images_dir, transforms=None):
        self.df = pd.read_csv(csv_file)
        self.images_dir = images_dir
        self.transforms = transforms
        self.image_ids = self.df['image_name'].unique()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df['image_name'] == image_id]

        image = Image.open(os.path.join(self.images_dir, image_id)).convert("RGB")
        boxes = []
        labels = []

        for _, row in records.iterrows():
            xmin = row['bbox_x']
            ymin = row['bbox_y']
            xmax = xmin + row['bbox_width']
            ymax = ymin + row['bbox_height']
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(1 if row['label_name'] == 'Marine Animal' else 2)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id

        # Convert image to tensor if no transformations are applied
        if self.transforms:
            image = self.transforms(image)
        else:
            image = T.ToTensor()(image)

        return image, target

# Define paths
csv_file = 'Dataset7.0/train/annotations/train_labels.csv'
images_dir = 'Dataset7.0/train/images'

# Create dataset and dataloaders without transformations
dataset = CustomCSVDetectionDataset(csv_file, images_dir, transforms=None)

# DataLoader setup
def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# Initialize the model
model = get_model(num_classes=3)  # 2 classes (marine animal, trash) + background
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Training loop
import torch.optim as optim

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

C:\Users\Ghodi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Ghodi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Define the path to your model weights
model_path = 'model/model_weights_epoch_43.pth'

# Initialize the model
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# Create an instance of the model
model = get_model(num_classes=3)  # Replace 3 with the number of classes in your dataset

# Load the model weights onto CPU
device = torch.device('cpu')  # Specify the device (CPU in this case)
model.load_state_dict(torch.load(model_path, map_location=device))

# Set the model to evaluation mode
model.eval()

# Print a message indicating successful loading
print(f"Model loaded from {model_path}")


C:\Users\Ghodi\AppData\Local\Temp\ipykernel_1520\1859032435.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

Model loaded from model/model_weights_epoch_43.pth


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class_colors = {
    1: (255, 165, 0),  # Marine Animal - Orange
    2: (255, 255, 0)   # Trash - Yellow
}

class_names = {
    1: 'Marine Animal',
    2: 'Trash'
}

label_to_int = {
    'Marine Animal': 1,
    'Trash': 2
}

In [6]:
def visualize_predictions(image, boxes, labels, scores, threshold=0.5):
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            box = box.astype(np.int32)
            color = class_colors.get(label, (255, 255, 255))
            label_name = class_names.get(label, 'Unknown')

            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color, 2)
            cv2.putText(image, f"{label_name}: {score:.2f}", (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [7]:
import streamlit as st
import cv2
import numpy as np
import torch
from torchvision.transforms import ToTensor
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision

# Function to load the model
def load_model(model_path):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    num_classes = 3  # Replace with the number of classes in your dataset
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load the model weights onto CPU
    device = torch.device('cpu')  # Use CPU for deployment
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Set the model to evaluation mode
    model.eval()

    return model

# Load the model
model_path = 'model/model_weights_epoch_43.pth'
model = load_model(model_path)

# Function to preprocess and predict
def predict_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image).unsqueeze(0)  # Convert to tensor
    with torch.no_grad():
        predictions = model(image_tensor)
    return predictions

# Streamlit app
def main():
    st.title('Marine Animals and Trash Detection')

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Read the uploaded image
        image = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), cv2.IMREAD_COLOR)

        # Perform prediction
        predictions = predict_image(image)

        # Example: Extract bounding boxes, labels, and scores from predictions
        pred_boxes = predictions[0]['boxes'].cpu().numpy()
        pred_labels = predictions[0]['labels'].cpu().numpy()
        pred_scores = predictions[0]['scores'].cpu().numpy()

        # Display the image with predictions
        st.image(image, channels="RGB", caption="Uploaded Image")

        # Visualize predictions on the image
        for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
            if score > 0.5:  # Adjust the threshold as needed
                st.markdown(f"**Label:** {label}, **Score:** {score:.2f}")
                image = cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

        # Display the annotated image
        st.image(image, channels="RGB", caption="Annotated Image with Predictions")

if __name__ == '__main__':
    main()


C:\Users\Ghodi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Ghodi\AppData\Local\Temp\ipykernel_1520\925296329.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to 

In [8]:
!streamlit run app.py

^C
